# TB Prediction System

In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
import os

class EnhancedTBPredictionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("TB Diagnostic System - Molecular Data Analysis")
        self.root.geometry("800x600")
        
        # Load all necessary components
        self.load_models_and_components()
        
        # Get final selected features from feature selectors
        self.final_features_TB_Status = self.get_final_selected_features('TB_Status')
        self.final_features_TB_Type = self.get_final_selected_features('TB_Type')
        self.final_features_TB_Stage = self.get_final_selected_features('TB_Stage')
        
        # Identify common features
        self.identify_common_features()
        
        self.create_enhanced_widgets()
    
    def load_models_and_components(self):
        """Load all trained models, scalers, feature selectors, and label encoders"""
        base_path = r'..\..\Saved_files'
        
        # Load models
        with open(os.path.join(base_path, 'best_sub_model_1.pkl'), 'rb') as file:
            self.model_TB_Status = pickle.load(file)
        with open(os.path.join(base_path, 'best_sub_model_2.pkl'), 'rb') as file:
            self.model_TB_Type = pickle.load(file)
        with open(os.path.join(base_path, 'best_sub_model_3.pkl'), 'rb') as file:
            self.model_TB_Stage = pickle.load(file)
        
        # Load scalers
        scaler_path = os.path.join(base_path, 'scalar')
        with open(os.path.join(scaler_path, 'scaler_TB_Status.pkl'), 'rb') as file:
            self.scaler_TB_Status = pickle.load(file)
        with open(os.path.join(scaler_path, 'scaler_TB_Type.pkl'), 'rb') as file:
            self.scaler_TB_Type = pickle.load(file)
        with open(os.path.join(scaler_path, 'scaler_TB_Stage.pkl'), 'rb') as file:
            self.scaler_TB_Stage = pickle.load(file)
        
        # Load feature selectors
        with open(os.path.join(base_path, 'feature_selector_TB_Status.pkl'), 'rb') as file:
            self.feature_selector_TB_Status = pickle.load(file)
        with open(os.path.join(base_path, 'feature_selector_TB_Type.pkl'), 'rb') as file:
            self.feature_selector_TB_Type = pickle.load(file)
        with open(os.path.join(base_path, 'feature_selector_TB_Stage.pkl'), 'rb') as file:
            self.feature_selector_TB_Stage = pickle.load(file)
        
        # Load label encoders
        with open(os.path.join(base_path, 'label_encoder_TB_Status.pkl'), 'rb') as file:
            self.label_encoder_TB_Status = pickle.load(file)
        with open(os.path.join(base_path, 'label_encoder_TB_Type.pkl'), 'rb') as file:
            self.label_encoder_TB_Type = pickle.load(file)
        with open(os.path.join(base_path, 'label_encoder_TB_Stage.pkl'), 'rb') as file:
            self.label_encoder_TB_Stage = pickle.load(file)
        
        # Load initial selected features (for reference)
        self.initial_features_TB_Status = pd.read_csv(
            os.path.join(base_path, 'selected_features_TB_HC_OD.csv')
        )['Selected Features'].tolist()
        self.initial_features_TB_Type = pd.read_csv(
            os.path.join(base_path, 'selected_features_PTB_EPTB.csv')
        )['Selected Features'].tolist()
        self.initial_features_TB_Stage = pd.read_csv(
            os.path.join(base_path, 'selected_features_ATB_LTB.csv')
        )['Selected Features'].tolist()
    
    def get_final_selected_features(self, model_type):
        """Get the final selected features after Extra Trees feature selection"""
        if model_type == 'TB_Status':
            selector = self.feature_selector_TB_Status
            initial_features = self.initial_features_TB_Status
        elif model_type == 'TB_Type':
            selector = self.feature_selector_TB_Type
            initial_features = self.initial_features_TB_Type
        elif model_type == 'TB_Stage':
            selector = self.feature_selector_TB_Stage
            initial_features = self.initial_features_TB_Stage
        else:
            return []
        
        # Get the feature mask from the selector
        feature_mask = selector.get_support()
        final_features = [feature for i, feature in enumerate(initial_features) if feature_mask[i]]
        
        return final_features
    
    def identify_common_features(self):
        """Identify common features across different submodels"""
        self.common_features = {
            'Status_Type': set(self.final_features_TB_Status) & set(self.final_features_TB_Type),
            'Status_Stage': set(self.final_features_TB_Status) & set(self.final_features_TB_Stage),
            'Type_Stage': set(self.final_features_TB_Type) & set(self.final_features_TB_Stage),
            'All_Three': set(self.final_features_TB_Status) & set(self.final_features_TB_Type) & set(self.final_features_TB_Stage)
        }
    
    def get_feature_description(self, feature):
        """Get description for a feature indicating which models use it"""
        if feature in self.common_features['All_Three']:
            return " (common for TB Status, Type, and Stage)"
        elif feature in self.common_features['Status_Type']:
            return " (common for TB Status and Type)"
        elif feature in self.common_features['Status_Stage']:
            return " (common for TB Status and Stage)"
        elif feature in self.common_features['Type_Stage']:
            return " (common for TB Type and Stage)"
        else:
            return ""
    
    def create_enhanced_widgets(self):
        """Create the enhanced GUI with both manual input and file upload options"""
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=1, padx=20, pady=20)
        
        # Title
        title_label = ttk.Label(main_frame, text="TB Diagnostic Prediction System", 
                               font=("Arial", 16, "bold"))
        title_label.pack(pady=10)
        
        # Mode selection
        mode_frame = ttk.LabelFrame(main_frame, text="Prediction Mode", padding=10)
        mode_frame.pack(fill=tk.X, pady=10)
        
        # Manual input button
        manual_btn = ttk.Button(mode_frame, text="Manual Input Mode", 
                               command=self.open_manual_input)
        manual_btn.pack(side=tk.LEFT, padx=5)
        
        # File upload button
        file_btn = ttk.Button(mode_frame, text="Upload Dataset File", 
                             command=self.upload_and_predict_dataset)
        file_btn.pack(side=tk.LEFT, padx=5)
        
        # Instructions for file upload
        instructions = ttk.Label(main_frame, 
                                text="Supported formats: CSV, TXT (GEO series matrix)\n"
                                     "File should contain gene expression data with ILMN_* features",
                                font=("Arial", 9), justify=tk.CENTER)
        instructions.pack(pady=10)
        
        # Results area
        self.results_text = tk.Text(main_frame, height=20, width=80, font=("Arial", 10))
        self.results_text.pack(fill=tk.BOTH, expand=1, pady=10)
        
        # Clear results button
        clear_btn = ttk.Button(main_frame, text="Clear Results", command=self.clear_results)
        clear_btn.pack(pady=5)
    
    def open_manual_input(self):
        """Open the manual input window organized by submodel"""
        manual_window = tk.Toplevel(self.root)
        manual_window.title("Manual Input - TB Diagnosis")
        manual_window.geometry("900x1000")
    
        # Create notebook for tabbed interface
        notebook = ttk.Notebook(manual_window)
        notebook.pack(fill=tk.BOTH, expand=1, padx=20, pady=20)
        
        # Create tabs for each submodel
        self.create_tb_status_tab(notebook)
        self.create_tb_type_tab(notebook)
        self.create_tb_stage_tab(notebook)
        
        # Prediction button
        predict_frame = ttk.Frame(manual_window)
        predict_frame.pack(fill=tk.X, padx=20, pady=10)
        
        predict_button = ttk.Button(predict_frame, text="Run Complete TB Diagnosis", 
                                  command=lambda: self.predict_from_manual(notebook))
        predict_button.pack(side=tk.LEFT, padx=5)
        
        close_button = ttk.Button(predict_frame, text="Close", command=manual_window.destroy)
        close_button.pack(side=tk.RIGHT, padx=5)
    
    def create_tb_status_tab(self, notebook):
        """Create tab for TB Status features"""
        status_frame = ttk.Frame(notebook)
        notebook.add(status_frame, text=f"TB Status ({len(self.final_features_TB_Status)} features)")
        
        self.create_feature_input_section(status_frame, self.final_features_TB_Status, "TB_Status")
    
    def create_tb_type_tab(self, notebook):
        """Create tab for TB Type features"""
        type_frame = ttk.Frame(notebook)
        notebook.add(type_frame, text=f"TB Type ({len(self.final_features_TB_Type)} features)")
        
        self.create_feature_input_section(type_frame, self.final_features_TB_Type, "TB_Type")
    
    def create_tb_stage_tab(self, notebook):
        """Create tab for TB Stage features"""
        stage_frame = ttk.Frame(notebook)
        notebook.add(stage_frame, text=f"TB Stage ({len(self.final_features_TB_Stage)} features)")
        
        self.create_feature_input_section(stage_frame, self.final_features_TB_Stage, "TB_Stage")
    
    def create_feature_input_section(self, parent_frame, features, model_type):
        """Create feature input section for a specific model type"""
        # Title
        title_label = ttk.Label(parent_frame, 
                               text=f"Input Gene Expression Values for {model_type} Prediction",
                               font=("Arial", 12, "bold"))
        title_label.pack(pady=10)
        
        # Create scrollable frame
        canvas_frame = ttk.Frame(parent_frame)
        canvas_frame.pack(fill=tk.BOTH, expand=1)
        
        canvas = tk.Canvas(canvas_frame)
        scrollbar = ttk.Scrollbar(canvas_frame, orient=tk.VERTICAL, command=canvas.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)
        canvas.configure(yscrollcommand=scrollbar.set)
        
        # Frame inside canvas
        scrollable_frame = ttk.Frame(canvas)
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        
        # Create entry fields for each feature
        feature_entries = {}
        for i, feature in enumerate(features):
            frame = ttk.Frame(scrollable_frame)
            frame.pack(fill=tk.X, padx=10, pady=2)
            
            # Feature label with description
            description = self.get_feature_description(feature)
            label_text = f"{feature}{description}"
            label = ttk.Label(frame, text=label_text, width=60, anchor="w")
            label.pack(side=tk.LEFT)
            
            entry = ttk.Entry(frame, width=15)
            entry.pack(side=tk.RIGHT)
            feature_entries[feature] = entry
        
        # Store feature entries in the parent frame for later access
        parent_frame.feature_entries = feature_entries
        
        # Update scroll region when frame size changes
        def configure_scroll_region(event):
            canvas.configure(scrollregion=canvas.bbox("all"))
        
        scrollable_frame.bind("<Configure>", configure_scroll_region)
    
    def predict_from_manual(self, notebook):
        """Collect input from all tabs and make prediction"""
        try:
            # Collect features from all tabs
            all_features = {}
            
            # Get features from TB Status tab
            status_frame = notebook.nametowidget(notebook.tabs()[0])
            for feature, entry in status_frame.feature_entries.items():
                value = entry.get()
                if not value:
                    messagebox.showerror("Input Error", f"Please enter a value for {feature} in TB Status tab")
                    return
                try:
                    all_features[feature] = float(value)
                except ValueError:
                    messagebox.showerror("Input Error", f"Invalid numeric value for {feature}")
                    return
            
            # Get features from TB Type tab
            type_frame = notebook.nametowidget(notebook.tabs()[1])
            for feature, entry in type_frame.feature_entries.items():
                value = entry.get()
                if not value:
                    messagebox.showerror("Input Error", f"Please enter a value for {feature} in TB Type tab")
                    return
                try:
                    all_features[feature] = float(value)
                except ValueError:
                    messagebox.showerror("Input Error", f"Invalid numeric value for {feature}")
                    return
            
            # Get features from TB Stage tab
            stage_frame = notebook.nametowidget(notebook.tabs()[2])
            for feature, entry in stage_frame.feature_entries.items():
                value = entry.get()
                if not value:
                    messagebox.showerror("Input Error", f"Please enter a value for {feature} in TB Stage tab")
                    return
                try:
                    all_features[feature] = float(value)
                except ValueError:
                    messagebox.showerror("Input Error", f"Invalid numeric value for {feature}")
                    return
            
            # Create input DataFrame
            input_features_df = pd.DataFrame([all_features])
            
            # Make prediction
            results = self.batch_predict(input_features_df)
            
            if results:
                self.display_single_result(results[0])
                
        except Exception as e:
            messagebox.showerror("Prediction Error", f"An error occurred during prediction:\n{str(e)}")
    
    def display_single_result(self, result):
        """Display single prediction result in a formatted way"""
        # Clear previous results
        self.results_text.delete(1.0, tk.END)
    
        result_text = f"🔬 TB DIAGNOSTIC RESULTS\n"
        result_text += "=" * 50 + "\n\n"
    
        # TB Status
        result_text += f"📊 TB Status: {result['TB_Status']}\n"
        result_text += f"   Confidence: {result['TB_Status_Confidence']*100:.1f}%\n\n"
    
        # TB Type (if available)
        if result['TB_Type'] != 'N/A':
            result_text += f"🦠 TB Type: {result['TB_Type']}\n"
            result_text += f"   Confidence: {result['TB_Type_Confidence']*100:.1f}%\n\n"
    
        # TB Stage (if available)
        if result['TB_Stage'] != 'N/A':
            result_text += f"📈 TB Stage: {result['TB_Stage']}\n"
            result_text += f"   Confidence: {result['TB_Stage_Confidence']*100:.1f}%\n\n"
    
        # Overall confidence
        confidences = [result['TB_Status_Confidence']]
        if result['TB_Type'] != 'N/A':
            confidences.append(result['TB_Type_Confidence'])
        if result['TB_Stage'] != 'N/A':
            confidences.append(result['TB_Stage_Confidence'])
    
        avg_confidence = sum(confidences) / len(confidences)
        result_text += f"⭐ Overall Confidence: {avg_confidence*100:.1f}%\n\n"
    
        # Interpretation
        result_text += "💡 Interpretation:\n"
        if result['TB_Status'] == 'Healthy Control':
            result_text += "No evidence of tuberculosis infection detected.\n"
        elif result['TB_Status'] == 'Other Disease':
            result_text += "Pattern suggests other pulmonary condition, not tuberculosis.\n"
        else:  # TB detected
            result_text += f"Tuberculosis detected - {result['TB_Type']}"
            if result['TB_Stage'] != 'N/A':
                result_text += f" in {result['TB_Stage']} stage"
            result_text += "\n"
    
        self.results_text.insert(1.0, result_text)
    
    def upload_and_predict_dataset(self):
        """Handle dataset file upload and batch prediction"""
        file_path = filedialog.askopenfilename(
            title="Select Dataset File",
            filetypes=[("CSV files", "*.csv"), ("Text files", "*.txt"), ("All files", "*.*")]
        )
        
        if not file_path:
            return
        
        try:
            # Process the uploaded file
            raw_dataset = self.process_uploaded_file(file_path)
            
            if raw_dataset is not None:
                # Perform batch predictions
                results = self.batch_predict(raw_dataset)
                
                # Display results
                self.display_batch_results(results, raw_dataset)
                
        except Exception as e:
            messagebox.showerror("Processing Error", f"Error processing file:\n{str(e)}")
    
    def process_uploaded_file(self, file_path):
        """Process uploaded GEO dataset file"""
        try:
            # Determine file type and read accordingly
            if file_path.endswith('.txt'):
                # Handle GEO series matrix format
                dataset = self.process_geo_series_matrix(file_path)
            else:
                # Handle CSV format
                dataset = pd.read_csv(file_path)
            
            # Validate dataset structure
            if not self.validate_dataset(dataset):
                return None
            
            # Preprocess the dataset (similar to your preprocessing pipeline)
            processed_dataset = self.preprocess_new_dataset(dataset)
            
            return processed_dataset
            
        except Exception as e:
            raise Exception(f"File processing failed: {str(e)}")
    
    def process_geo_series_matrix(self, file_path):
        """Process GEO series matrix file format"""
        try:
            # Find where the data section starts
            with open(file_path, 'r') as file:
                lines = file.readlines()
            
            # Find the start of data section (skip metadata)
            data_start_line = 0
            for i, line in enumerate(lines):
                if line.startswith('!series_matrix_table_begin'):
                    data_start_line = i + 1
                    break
                if not line.startswith('!'):
                    data_start_line = i
                    break
            
            # Read the data section
            dataset = pd.read_csv(file_path, sep='\t', skiprows=data_start_line)
            
            # Transpose if necessary (genes as columns)
            if 'ID_REF' in dataset.columns:
                dataset = dataset.set_index('ID_REF').T
                dataset.index.name = 'Sample'
            
            return dataset
            
        except Exception as e:
            raise Exception(f"GEO file processing error: {str(e)}")
    
    def validate_dataset(self, dataset):
        """Validate that the dataset has the required structure"""
        # Check if dataset has any ILMN_ features
        ilmn_features = [col for col in dataset.columns if col.startswith('ILMN_')]
        
        if len(ilmn_features) == 0:
            messagebox.showerror("Validation Error", 
                               "No ILMN_ features found in the dataset.\n"
                               "Please ensure the file contains gene expression data from Illumina platform.")
            return False
        
        # Check for numeric data
        numeric_check = dataset[ilmn_features[:5]].apply(pd.to_numeric, errors='coerce').notna().all().all()
        if not numeric_check:
            messagebox.showerror("Validation Error", 
                               "Dataset contains non-numeric values in gene expression columns.")
            return False
        
        return True
    
    def preprocess_new_dataset(self, dataset):
        """Preprocess new dataset to match training data format"""
        # Select only ILMN_ features
        ilmn_features = [col for col in dataset.columns if col.startswith('ILMN_')]
        processed_data = dataset[ilmn_features].copy()
        
        # Convert to numeric
        processed_data = processed_data.apply(pd.to_numeric, errors='coerce')
        
        # Handle missing values (fill with mean)
        processed_data = processed_data.fillna(processed_data.mean())
        
        return processed_data
    
    def handle_missing_features(self, input_features):
        """Handle missing features by filling with zeros"""
        # Combine all required features
        all_required_features = (set(self.final_features_TB_Status) | 
                               set(self.final_features_TB_Type) | 
                               set(self.final_features_TB_Stage))
        
        missing_features = all_required_features - set(input_features.columns)
        
        if missing_features:
            print(f"Warning: {len(missing_features)} features missing. Filling with zeros.")
            for feature in missing_features:
                input_features[feature] = 0.0
        
        # Ensure all required features are present
        input_features = input_features.reindex(columns=list(all_required_features), fill_value=0.0)
        
        return input_features
    
    def batch_predict(self, input_dataset):
        """Perform batch predictions on multiple samples using only final selected features"""
        results = []
        
        # Handle missing features
        input_dataset_processed = self.handle_missing_features(input_dataset)
        
        for sample_idx, sample_data in input_dataset_processed.iterrows():
            try:
                sample_result = self.predict_single_sample(sample_data)
                sample_result['Sample_ID'] = sample_idx
                results.append(sample_result)
            except Exception as e:
                print(f"Error predicting sample {sample_idx}: {str(e)}")
                results.append({
                    'Sample_ID': sample_idx,
                    'TB_Status': 'Error',
                    'TB_Status_Confidence': 0,
                    'TB_Type': 'N/A',
                    'TB_Type_Confidence': 0,
                    'TB_Stage': 'N/A',
                    'TB_Stage_Confidence': 0,
                    'Error': str(e)
                })
        
        return results
    
    def predict_single_sample(self, sample_data):
        """Predict for a single sample using only final selected features"""
        # Convert sample to DataFrame
        input_features = pd.DataFrame([sample_data])
        
        # Transform for TB_Status using only final features
        X_status_input = input_features[self.final_features_TB_Status]
        input_features_TB_Status = self.scaler_TB_Status.transform(X_status_input)
        
        TB_Status_pred_encoded = self.model_TB_Status.predict(input_features_TB_Status)
        TB_Status_prob = self.model_TB_Status.predict_proba(input_features_TB_Status)
        TB_Status_pred = self.label_encoder_TB_Status.inverse_transform(TB_Status_pred_encoded)[0]
        TB_Status_confidence = max(TB_Status_prob[0])
        
        result = {
            'TB_Status': TB_Status_pred,
            'TB_Status_Confidence': TB_Status_confidence,
            'TB_Type': 'N/A',
            'TB_Type_Confidence': 0,
            'TB_Stage': 'N/A',
            'TB_Stage_Confidence': 0
        }
        
        # If TB predicted, proceed to type classification
        if TB_Status_pred == 'TB':
            # Transform for TB_Type using only final features
            X_type_input = input_features[self.final_features_TB_Type]
            input_features_TB_Type = self.scaler_TB_Type.transform(X_type_input)
            
            TB_Type_pred_encoded = self.model_TB_Type.predict(input_features_TB_Type)
            TB_Type_prob = self.model_TB_Type.predict_proba(input_features_TB_Type)
            TB_Type_pred = self.label_encoder_TB_Type.inverse_transform(TB_Type_pred_encoded)[0]
            TB_Type_confidence = max(TB_Type_prob[0])
            
            result['TB_Type'] = TB_Type_pred
            result['TB_Type_Confidence'] = TB_Type_confidence
            
            # If PTB or EPTB predicted, proceed to stage classification
            if TB_Type_pred in ['Pulmonary TB', 'Extra Pulmonary TB']:
                # Transform for TB_Stage using only final features
                X_stage_input = input_features[self.final_features_TB_Stage]
                input_features_TB_Stage = self.scaler_TB_Stage.transform(X_stage_input)
                
                TB_Stage_pred_encoded = self.model_TB_Stage.predict(input_features_TB_Stage)
                TB_Stage_prob = self.model_TB_Stage.predict_proba(input_features_TB_Stage)
                TB_Stage_pred = self.label_encoder_TB_Stage.inverse_transform(TB_Stage_pred_encoded)[0]
                TB_Stage_confidence = max(TB_Stage_prob[0])
                
                result['TB_Stage'] = TB_Stage_pred
                result['TB_Stage_Confidence'] = TB_Stage_confidence
        
        return result
    
    def display_batch_results(self, results, original_dataset):
        """Display batch prediction results in a formatted way"""
        self.results_text.delete(1.0, tk.END)
        
        result_text = "🔬 BATCH TB DIAGNOSTIC RESULTS\n"
        result_text += "=" * 60 + "\n\n"
        result_text += f"📊 Total Samples Processed: {len(results)}\n\n"
        
        # Create a summary table
        result_text += "Sample ID".ljust(15) + "TB Status".ljust(40) + "TB Type".ljust(40) + "TB Stage".ljust(40) + "Confidence\n"
        result_text += "-" * 180 + "\n"
        
        for result in results:
            sample_id = str(result['Sample_ID'])[:18].ljust(20)
            status = result['TB_Status'].ljust(40)
            tb_type = result['TB_Type'].ljust(40)
            tb_stage = result['TB_Stage'].ljust(40)
            
            # Calculate overall confidence
            confidences = [result['TB_Status_Confidence']]
            if result['TB_Type'] != 'N/A':
                confidences.append(result['TB_Type_Confidence'])
            if result['TB_Stage'] != 'N/A':
                confidences.append(result['TB_Stage_Confidence'])
            
            avg_confidence = sum(confidences) / len(confidences)
            
            result_text += f"{sample_id}{status}{tb_type}{tb_stage}{avg_confidence:.1%}\n"
        
        # Add summary statistics
        result_text += "\n" + "=" * 60 + "\n"
        result_text += "📈 SUMMARY STATISTICS:\n"
        
        status_counts = {}
        type_counts = {}
        stage_counts = {}
        
        for result in results:
            status = result['TB_Status']
            tb_type = result['TB_Type']
            tb_stage = result['TB_Stage']
            
            status_counts[status] = status_counts.get(status, 0) + 1
            if tb_type != 'N/A':
                type_counts[tb_type] = type_counts.get(tb_type, 0) + 1
            if tb_stage != 'N/A':
                stage_counts[tb_stage] = stage_counts.get(tb_stage, 0) + 1
        
        result_text += f"\nTB Status Distribution:\n"
        for status, count in status_counts.items():
            result_text += f"  {status}: {count} samples ({count/len(results):.1%})\n"
        
        if type_counts:
            result_text += f"\nTB Type Distribution:\n"
            for tb_type, count in type_counts.items():
                result_text += f"  {tb_type}: {count} samples\n"
        
        if stage_counts:
            result_text += f"\nTB Stage Distribution:\n"
            for tb_stage, count in stage_counts.items():
                result_text += f"  {tb_stage}: {count} samples\n"
        
        self.results_text.insert(1.0, result_text)
    
    def clear_results(self):
        """Clear the results text area"""
        self.results_text.delete(1.0, tk.END)

# Run the enhanced app
if __name__ == "__main__":
    root = tk.Tk()
    app = EnhancedTBPredictionApp(root)
    root.mainloop()

C:\Users\dilan\AppData\Local\Temp\ipykernel_31512\2466795024.py:478: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_features[feature] = 0.0
C:\Users\dilan\AppData\Local\Temp\ipykernel_31512\2466795024.py:478: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_features[feature] = 0.0
c:\Users\dilan\anaconda3\envs\myenv\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\dilan\anaconda3\envs\myenv\lib\site-packages\sklearn\base